---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

In [138]:
dict_1 = { 'dict1' : [1, 2, 3], 'dict2' : 'string' }
type(dict_1.keys())

dict_keys

In [142]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

twenty_train.target[:10]

for t in twenty_train.target[:10] :
    print(twenty_train['target_names'][t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


### Extracting features from text files

In order to perform machine learning on text documents, we first need to turn the text content into numerical vectors.

#### - Bags of words

The most intuitive way to do so is the bags of words representation :
1. Assign a fixed integer id to each word occurring in any document of the training set (by building a dictionary from words to integer indices).<br><br>
2. For each document `#i`, count the number of occurrences of each word `w` and store it in `X[i, j]` as the value of feature `#j` where `j` is the index of word `w` in the dictionary.

It implies that 'n_features' is the number of distinct words in the corpus.

If `n_samples == 10000`, storing X as a numpy array of type float32 would require 10000 * 100000 * 4 bytes = 4GB in RAM.

Fortunately, ** most values in X will be zeros**. So we say that bags of words are typically **high-dimensional sparse datasets.** 

<br>
#### - Tokenizing text with scikit-learn

Text processing is included in a high level component that is able to build a dictionary of features and transform documents to feature vectors :

**CountVectorizer** supports counts of N-grams of words or consecutive characters. 

Once fitted, the vectorizer has built a dictionary of feature indices. 

<br>


In [143]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [145]:
count_vect.vocabulary_.get(u'algorithm')

4690

#### - From occurrences to frequencies

Occurrence count는 좋은 방법이지만 한 가지 문제가 있다.

-> 문서의 분량이 많으면, 같은 주제에 대해 이야기하더라도 평균 count 값이 다를 수 있다는 점이다.

-> 이러한 경우, count를 전체 문서의 단어 개수로 나눠주면 된다. 

-> 이렇게 나눠준 값을 ** tf (Term Frequency) ** 라고 한다.


<br>
또한 여러 문서에서 자주 나타나는 단어(정보가 적은 단어)의 가중치를 낮춰줘야 한다. 

-> 이를 ** tf-idf (Term Frequency times Inverse Document Frequency** 라고 한다.


In [147]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [149]:
# Same as above, but (fit, transform) altogether
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

<br>
#### - Training a classifier

Now that we have our features, we can train a classifier to try to predict the category of a post.

Let's start with a **Naive Bayes Classifier** . We will use word counts in multinomial variant.


In [150]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [75]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
print(spam_data.values)
spam_data.head()

[[ 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
  0]
 ['Ok lar... Joking wif u oni...' 0]
 [ "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
  1]
 ..., 
 ['Pity, * was in mood for that. So...any other suggestions?' 0]
 [ "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"
  0]
 ['Rofl. Its true to its name' 0]]


,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [48]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [67]:
X_train

872                       I'll text you when I drop x off
831     Hi mate its RV did u hav a nice hol just a mes...
1273    network operator. The service is free. For T &...
3314    FREE MESSAGE Activate your 500 FREE Text Messa...
4929    Hi, the SEXYCHAT girls are waiting for you to ...
4249                              How much for an eighth?
3640    You can stop further club tones by replying \S...
1132                  Good morning princess! How are you?
3318                     Kay... Since we are out already 
5241                            Its a part of checking IQ
3534                             I'm at home. Please call
1380                               i dnt wnt to tlk wid u
526     Today's Offer! Claim ur å£150 worth of discoun...
2448                         Do u knw dis no. &lt;#&gt; ?
4029                         Cool, I'll text you in a few
641      said kiss, kiss, i can't do the sound effects...
5570    The guy did some bitching but I acted like i'd...
5111          

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [18]:
def answer_one():
    
    
    return spam_data[ spam_data['target']==1].shape[0] / float(spam_data.shape[0]) * 100

spam_data.index

RangeIndex(start=0, stop=5572, step=1)

In [15]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(X_train)
    
    analyzer = vectorizer.build_analyzer()
    print (X.toarray())
    pass
answer_two()

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [ ]:
answer_two()

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    
    return #Your answer here

In [ ]:
answer_three()

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    
    return #Your answer here

In [ ]:
answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
def answer_five():
    
    
    return #Your answer here

In [ ]:
answer_five()

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [ ]:
def answer_six():
    
    
    return #Your answer here

In [ ]:
answer_six()

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [ ]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.svm import SVC

def answer_seven():
    
    
    return #Your answer here

In [ ]:
answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [ ]:
def answer_eight():
    
    
    return #Your answer here

In [ ]:
answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    
    return #Your answer here

In [ ]:
answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [ ]:
def answer_ten():
    
    
    return #Your answer here

In [ ]:
answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [ ]:
def answer_eleven():
    
    
    return #Your answer here

In [ ]:
answer_eleven()